# FMEA Example

In [1]:
import os
import pandas as pd
import numpy as np
import sys
sys.path.append(os.path.join("..","..","..",".."))
from mika.kd.NER import *
from mika.kd import FMEA 
from torch import cuda

## Define Severity Calculation

To get a severity column, define a severity function that creates a numeric value of severity from the data

In [2]:
def calc_severity(df):
    severities = []
    for i in range(len(df)):
        severities.append(safecom_severity(df.iloc[i]['Hazardous Materials'], df.iloc[i]['Injuries'], df.iloc[i]['Damages']))
    df['severity'] = severities
    return df

def safecom_severity(hazardous_mat, injury, damage):
    key_dict = {"No":0, "Yes":1}
    severity = key_dict[hazardous_mat] + key_dict[injury] + key_dict[damage]
    if np.isnan(severity):
        severity=0
    return severity

## Initiate FMEA 
- create object
- load model
- load data

In [3]:
model_checkpoint = "NASA-AIML/MIKA_BERT_FMEA_NER"

In [4]:
fmea = FMEA()
fmea.load_model(model_checkpoint)
file = os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir, os.pardir, os.pardir)),"data/SAFECOM/SAFECOM_UAS_fire_data.csv")
input_data = fmea.load_data(filepath=file, formatted=False, text_col='Text', id_col='Tracking #')


## Predict Entities

In [5]:
preds = fmea.predict()
df = fmea.get_entities_per_doc()

## Post Process

In [6]:
manual_cluster_file = os.path.join(os.getcwd(),"SAFECOM_UAS_clusters_V1.xlsx")
fmea.group_docs_manual(manual_cluster_file, grouping_col='Mode', additional_cols=['Mission Type'])
fmea.calc_severity(calc_severity)
fmea.get_year_per_doc('Date')
fmea.calc_frequency('Year')
fmea.calc_risk()
fmea.post_process_fmea(id_name='SAFECOM', max_words=10)

,Phase,Cause,Failure Mode,Effect,Control Process,Recommendations,Frequency,Severity,Risk,SAFECOM
cluster,,,,,,,,,,
Battery,Reconnaissance; Infrared Imagery,"“ b, button, not, could, issue, battery level ...","hard, landing, depleted, battery, 40 percent, ...","dropping, 10, percent, lost, fell at close to ...","assumed manual, control, bringing it down, man...","batteries will be, tracked on an individual, l...",2,0.333333,0.666667,19-0295
Casing dislodged,Reconnaissance; Infrared Imagery,"fuselage, cowling on the aircraft was, missing...","battery hatch cover disengaged from the, motor...","hit, motors, shut down, the, small, nick, debr...","propeller and battery hatch cover were, replac...",replace,2,0.000000,0.000000,21-0015
Communications/flight navigation with UAS failed,Infrared Imagery; External Load (Longline),"no contact with, operation, not know about, mi...","positive contact with a, uas operation, failed...",flight navigation software on the computer sta...,"confirmed frequencies and mission, parameters,...","separation, contact, gain, approvals, and plan...",2,0.000000,0.000000,17-0971
Employee personal drone usage without certification/official approval,Initial Attack; Reconnaissance; Aerial Ignitio...,"did not recall any discussion of uas, use, nan...","ua, was put, the, litter","away, not, consuming, needle","1420 acre prescribed, burn, uas was, aerial, p...","check the fire, activity, the, fuels, size of,...",2,0.000000,0.000000,19-0448
Engine Failure,Other; Infrared Imagery,"remained in guided, mode, attempting to mainta...","a33n engine, failed, aircraft, failure of the,...","ua, ) was, aircraft then, spiral, and, crashed...","aircraft, landing, monitoring the, visually, c...","with another, replace current mission planner,...",3,0.500000,1.500000,20-0822
Fight plan error,Reconnaissance; Infrared Imagery; Air Attack,"location of the, fire, placement of the launch...","uas aggressively changed its angle of, attack,...","ground, sustained, significant, damage, higher...","heat, flight, safety and operational, briefing...","detect any, remaining, map their, perimeters, ...",3,0.500000,1.500000,20-1303
Hang Fire,Aerial Ignition,"form of, visible, hang fire, functioned, melte...","a, hang, fire, on, aircraft gave a, hatch moti...","in, flight, fires, fire","vo assisted the pilot, resetting the ignis per...","follow, immediately, using the, camera, ensure...",1,0.000000,0.000000,20-0872
High wind pushes UAS off course,Reconnaissance,,"propeller, strike, side gust of, hit a, landin...","wind, off, target, ground was, struck, breakin...","in guided, mode, automated rtl, landing, area,...","relocate, larger, area, clear of all, obstacle...",2,0.500000,1.000000,18-0847
Hobbyist offers services,Initial Attack,"mission, no, t",,,"fire, department, camera - equipped, drone, wa...","fly, will not be, allowed, faa, accreditation,...",1,0.000000,0.000000,15-0166
